In [3]:
import openai
import pandas as pd
import os

In [4]:
apikey_path = 'apikey.txt'#读取同文件夹目录的文件
with open(apikey_path, 'r') as f:
    key = f.read()

#连接chatgpt
openai.organization = "org-tCd0A0x7hh3O9EpDr9OiV0Or" #它用于标识具有访问权限的 OpenAI 组织，大部分情况不需要
openai.api_key = key#api访问接口

1 加载数据 csv->dataframe

In [4]:
# 加载同文件下的csv文件
df = pd.read_csv('data.csv')

# 将第一列命名为prompt，第二列命名为completion
df.columns = ['prompt', 'completion']

In [5]:
df

,prompt,completion
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative
...,...,...
579,I have to cancel my vacation plans because I c...,negative
580,My computer crashed and I lost all my importan...,negative
581,I got into a car accident and my car is totale...,negative
582,I have a cold and can't stop coughing. it's re...,negative


2 格式转换 dataframe->jsonl

In [6]:
'''
目的是数据格式转换
dataframe->jsonl
文件保存在本地
'''
df.to_json(
    "data.jsonl", #存放地址和名称
    orient='records', #每个记录作为一行
    lines=True#每一行都是一个{a,b}的格式
    )

3 数据质量优化

In [7]:
'''
使用数据准备工具
目的是提高输入数据质量。智能增加提示符。并且将数据划分为训练集和测试集。
jsonl->jsonl
文件保存在本地
!openai tools fine_tunes.prepare_data -f 文件名.jsonl -q
-q代表接受所有建议和调整
'''

!openai tools fine_tunes.prepare_data -f data.jsonl -q

Analyzing...

- Your file contains 584 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 38 duplicated prompt-completion sets. These are rows: [153, 159, 164, 166, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 187, 193, 196, 203, 205, 215, 223, 228, 230, 247, 251, 253, 254, 256, 257, 261, 262, 268, 270, 276, 277, 289, 302, 306]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, 

D:\anaconda3\lib\site-packages\openai\validators.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["prompt"] += suffix
D:\anaconda3\lib\site-packages\openai\validators.py:421: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["completion"] = x["completion"].apply(


In [8]:
'''
翻译以上文本内容：

[建议]删除38个重复行 [Y/n]：Y
[建议]将后缀分隔符->添加到所有提示中 [Y/n]：Y
[建议]在完成的开头添加一个空格字符 [Y/n]：Y
[建议]您是否想要分成训练集和验证集？ [Y/n]：Y

建议运行这行代码来开启fine tuning，需要自己添加指定模型
openai api fine_tunes.create -t "data_prepared_train.jsonl" -v "data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 3

实际使用时候，prompt必须以->结尾来提示模型开始工作
'''

'\n翻译以上文本内容：\n\n[建议]删除38个重复行 [Y/n]：Y\n[建议]将后缀分隔符->添加到所有提示中 [Y/n]：Y\n[建议]在完成的开头添加一个空格字符 [Y/n]：Y\n[建议]您是否想要分成训练集和验证集？ [Y/n]：Y\n\n建议运行这行代码来开启fine tuning，需要自己添加指定模型\nopenai api fine_tunes.create -t "data_prepared_train.jsonl" -v "data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 3\n\n实际使用时候，prompt必须以->结尾来提示模型开始工作\n'

4 开始fine tuning

In [7]:
os.environ["OPENAI_API_KEY"] = key
'''
key是一个str数据类型
将你的key设置到环境变量中
如果不这样做，fine tuning将报错说没有提供key
'''

In [8]:
#模型训练
'''
!openai api fine_tunes.create
-t"本地训练数据集地址" 
-v"本地验证数据集地址"
--compute_classification_metrics           (非必须的参数。用于告诉微调进程计算与分类任务相关的度量标准，如准确率、精确度和召回率等。
--classification_n_classes 3               (非必须的参数。在多分类问题中，指定分类任务的类别总数量。
--classification_positive_class "类别名1"  (非必须的参数。在二分类问题中，指定正类别。
-m ada 指定模型，不需要加双引号
--suffix"命名模型名称"
'''


!openai api fine_tunes.create -t "data_prepared_train.jsonl" -v "data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 3 -m ada


Uploaded file from data_prepared_train.jsonl: file-F64kjWk9WI88Dp3hUpHjLThj
Uploaded file from data_prepared_valid.jsonl: file-AmUSj6jo8WdbfcKCP4ZVIBbl
Created fine-tune: ft-A8yZ7i95eWDdm6rmsiXGtFJ0
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-04-28 13:13:24] Created fine-tune: ft-A8yZ7i95eWDdm6rmsiXGtFJ0
[2023-04-28 13:13:34] Fine-tune costs $0.01
[2023-04-28 13:13:35] Fine-tune enqueued. Queue number: 0
[2023-04-28 13:13:37] Fine-tune started




Upload progress:   0%|          | 0.00/35.1k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 35.1k/35.1k [00:00<?, ?it/s]

Upload progress:   0%|          | 0.00/9.20k [00:00<?, ?it/s]
Upload progress: 100%|██████████| 9.20k/9.20k [00:00<?, ?it/s]


5 查看结果

In [10]:
ft="ft-A8yZ7i95eWDdm6rmsiXGtFJ0"

In [12]:
!openai api fine_tunes.follow -i $ft

[2023-04-28 13:13:24] Created fine-tune: ft-A8yZ7i95eWDdm6rmsiXGtFJ0
[2023-04-28 13:13:34] Fine-tune costs $0.01
[2023-04-28 13:13:35] Fine-tune enqueued. Queue number: 0
[2023-04-28 13:13:37] Fine-tune started
[2023-04-28 13:14:58] Completed epoch 1/4
[2023-04-28 13:16:09] Completed epoch 2/4
[2023-04-28 13:17:17] Completed epoch 3/4
[2023-04-28 13:18:25] Completed epoch 4/4
[2023-04-28 13:18:51] Uploaded model: ada:ft-personal-2023-04-28-05-18-51
[2023-04-28 13:18:52] Uploaded result file: file-vLYHmpeK6id6SEPkGDTuhEqT
[2023-04-28 13:18:52] Fine-tune succeeded


Traceback (most recent call last):
  File "D:\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "D:\anaconda3\Scripts\openai.exe\__main__.py", line 7, in <module>
  File "D:\anaconda3\lib\site-packages\openai\_openai_scripts.py", line 70, in main
    args.func(args)
  File "D:\anaconda3\lib\site-packages\openai\cli.py", line 516, in follow
    cls._stream_events(args.id)
  File "D:\anaconda3\lib\site-packages\openai\cli.py", line 559, in _stream_events
    sys.stdout.write("\nJob complete! Status: succeeded \U0001f389")
UnicodeEncodeError: 'gbk' codec can't encode character '\U0001f389' in position 34: illegal multibyte sequence


6 测试模型

In [27]:
#包装成一个方法
def generate_text(prompt):
    response = openai.Completion.create(
        engine='ada:ft-personal-2023-04-28-05-18-51',
        prompt=prompt,
        max_tokens=1,#机器人回复长度
        temperature=0.5,
        n=1,
        stop=None,
    )
    message = response.choices[0].text
    return message

In [28]:
#实现连续请求
while True:
    prompt = input("You: ")+"->"   #输入对话，默认这里加入命令符
    if prompt.lower() == "bye->":#如果输入bye就结束对话
        break
    response = generate_text(prompt)
    print("Chatbot:", response)

You: 我很开心
Chatbot:  positive
You: 我很难过
Chatbot:  negative
You: bye


7 查看训练过程参数

In [29]:
#因为刚刚这行代码--compute_classification_metrics 告诉要计算相应的指标
#下载这个指标
!openai api fine_tunes.results -i ft-A8yZ7i95eWDdm6rmsiXGtFJ0 > result.csv

In [32]:
#查看结果
results = pd.read_csv('result.csv')
results

,step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/weighted_f1_score
0,1,25,1,0.554468,0.0,0.0,0.356435,0.0,0.0,NaN,NaN
1,2,34,2,1.149400,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,3,43,3,1.122204,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,4,52,4,1.228593,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,5,69,5,0.341322,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1741,1742,25206,1742,0.008853,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1742,1743,25215,1743,0.008690,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1743,1744,25232,1744,0.003998,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1744,1745,25241,1745,0.006900,1.0,1.0,0.010964,1.0,1.0,NaN,NaN
